# Наумкин Владимир, С01-119.

## Задание 2. Анализ модели LSTM.

### Подключим библиотеки

In [1]:
from tqdm.notebook import tqdm
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from prettytable import PrettyTable
from nerus import load_nerus

### Уберём предупреждения

In [2]:
import warnings
warnings.filterwarnings("ignore")

### Зададим устройство исполнения кода (вычисления провожу на своём ПК)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

### Функции для работы с датасетом

In [4]:
def get_sent_tags(docs, size=10000):
    list_of_sent = []
    list_of_tags = []
    for doc in tqdm(docs):
        for sent in doc.sents:
            list_of_sent_toks = []
            list_of_sent_tags = []
            for tok in sent.tokens:
                list_of_sent_toks.append(tok.text)
                list_of_sent_tags.append(tok.pos)
        list_of_sent.append(list_of_sent_toks)
        list_of_tags.append(list_of_sent_tags)
        if len(list_of_sent) > size:
            break
    return list_of_sent, list_of_tags

In [5]:
def pos_dict(list_of_tags, test_size=100):
    pos2idx = {'<PAD>' : 0}
    idx2pos = ['<PAD>']
    for tags in list_of_tags[:-test_size]:
        for word in tags:
            if word not in pos2idx:
                pos2idx[word] = len(idx2pos)
                idx2pos.append(word)
    return pos2idx, idx2pos

In [6]:
def word_dict(list_of_tags, test_size=100):
    word2idx = {'<PAD>': 0, '<UNK>': 1}
    idx2word = ['<PAD>', '<UNK>']
    for sent in list_of_sent[:-test_size]:
        for word in sent:
            if word not in word2idx:
                word2idx[word] = len(idx2word)
                idx2word.append(word)
    return word2idx, idx2word

In [7]:
class NerusDataset(Dataset):
    def __init__(self, list_of_sent, list_of_tags, word2idx, pos2idx, train=True, test_size=100):
        self.X = []
        self.y = []
        if train:
            for sent in list_of_sent[:-test_size]:
                data = []
                for word in sent:
                    data.append(word2idx.get(word, 0))
                self.X.append(data)
            for tags in list_of_tags[:-test_size]:
                data = []
                for word in tags:
                    data.append(pos2idx.get(word, 0))
                self.y.append(data)
        else:
            for sent in list_of_sent[-test_size:]:
                data = []
                for word in sent:
                    data.append(word2idx.get(word, 0))
                self.X.append(data)
            for tags in list_of_tags[-test_size:]:
                data = []
                for word in tags:
                    data.append(pos2idx.get(word, 0))
                self.y.append(data)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return torch.Tensor(self.X[idx]), torch.Tensor(self.y[idx])

In [8]:
def collate_fn(data):
    X, Y = [], []
    for x, y in data:
        X.append(x)
        Y.append(y)
    x_batch = torch.zeros((len(X), max(list(map(len, X)))), dtype=torch.long)
    y_batch = torch.zeros((len(Y), max(list(map(len, Y)))), dtype=torch.long)
    for i, sent in enumerate(X):
        x_batch[i, :len(sent)] = sent
    for i, sent in enumerate(Y):
        y_batch[i, :len(sent)] = sent
    return x_batch, y_batch

### Функция проверки качества модели

In [9]:
def check_model(batch_size, dataset, model, loss_function, idx2word, idx2pos):
    model.eval()
    batch_generator = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, collate_fn=collate_fn)
    test_acc = 0
    test_loss = 0
    count = 0
    for it, (x_batch, y_batch) in enumerate(batch_generator):
        x_batch = x_batch.to(model.device)
        y_batch = y_batch.to(model.device)
        mask = (y_batch != 0)
        count += mask.sum()
        with torch.no_grad():
            output = model(x_batch)
        test_loss += loss_function(output.transpose(1,2), y_batch).cpu().item()*len(x_batch)
        test_acc += (torch.argmax(output, dim=-1).cpu() == y_batch)[mask].sum().item()
    test_loss /= len(dataset)
    test_acc /= count
    print(f'loss: {test_loss}, acc: {test_acc}')
    dataloader = torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)
    x, y = next(iter(dataloader))
    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        outputs = model(x)
    one_x = x[0].cpu().numpy()
    one_y = y[0].cpu().numpy()
    one_output = outputs[0].argmax(dim=-1).cpu().numpy()
    words = [idx2word[idx] for idx in one_x]
    true_tags = [idx2pos[idx] for idx in one_y]
    pred_tags = [idx2pos[idx] for idx in one_output]
    table = PrettyTable(["Word", "True tag", "Predicted tag"])
    table.align["Word"], table.align["True tag"], table.align["Predicted tag"] = "l", "l", "l"
    for word, true_tag, pred_tag in zip(words, true_tags, pred_tags):
        if word != idx2word[word2idx['<PAD>']]:
            table.add_row([word, true_tag, pred_tag])
    print(table)
    return test_loss, test_acc

### Код для обучения модели

In [10]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    model.zero_grad()
    output = model(x_batch.to(device))
    loss = loss_function(output.transpose(1, 2), y_batch.to(device))
    loss.backward()
    optimizer.step()
    return loss.cpu().item()

In [11]:
def train_epoch(train_generator, model, loss_function, optimizer, callback = None):
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x.to(device), batch_of_y.to(device), optimizer, loss_function)
        train_generator.set_postfix({'train batch loss': batch_loss})
        if callback is not None:
            callback(model, batch_loss)
        epoch_loss += batch_loss*len(batch_of_x)
        total += len(batch_of_x)
    return epoch_loss/total

In [12]:
def trainer(count_of_epoch, 
            batch_size, 
            dataset,
            model, 
            loss_function,
            optimizer,
            lr = 0.001,
            callback = None):
    optima = optimizer(model.parameters(), lr=lr)
    iterations = tqdm(range(count_of_epoch), desc='epoch')
    iterations.set_postfix({'train epoch loss': np.nan})
    for it in iterations:
        batch_generator = tqdm(
            torch.utils.data.DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn), 
            leave=False, total=len(dataset)//batch_size+(len(dataset)%batch_size> 0))
        epoch_loss = train_epoch(
                    train_generator=batch_generator, 
                    model=model, 
                    loss_function=loss_function, 
                    optimizer=optima, 
                    callback=callback)
        iterations.set_postfix({'train epoch loss': epoch_loss})

### Отслеживание обучения

In [13]:
class callback():
    def __init__(self, writer, dataset, loss_function, delimeter = 300, batch_size=64):
        self.step = 0
        self.writer = writer
        self.delimeter = delimeter
        self.loss_function = loss_function
        self.batch_size = batch_size
        self.dataset = dataset
    def forward(self, model, loss):
        self.step += 1
        self.writer.add_scalar('LOSS/train', loss, self.step)
        if self.step % self.delimeter == 0:
            model.eval()
            batch_generator = torch.utils.data.DataLoader(dataset=self.dataset,
                                                          batch_size=self.batch_size,
                                                          collate_fn=collate_fn)
            test_acc = 0
            test_loss = 0
            count = 0
            for it, (x_batch, y_batch) in enumerate(batch_generator):
                x_batch = x_batch.to(model.device)
                y_batch = y_batch.to(model.device)
                mask = (y_batch != 0)
                count += mask.sum()
                with torch.no_grad():
                    output = model(x_batch)
                test_loss += self.loss_function(output.transpose(1,2), y_batch).cpu().item()*len(x_batch)
                test_acc += (torch.argmax(output, dim=-1).cpu() == y_batch)[mask].sum().item()
            test_loss /= len(self.dataset)
            test_acc /= count
            print(f"\t step={self.step}, train_loss={loss}, val_loss={test_loss}, val_acc={test_acc}")
            self.writer.add_scalar('LOSS/test', test_loss, self.step)
            self.writer.add_scalar('ACC/test', test_acc, self.step)
    def __call__(self, model, loss):
        return self.forward(model, loss)

## Модель LSTM

In [14]:
class LSTM(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
    def __init__(self,
                 vocab_dim,
                 output_dim = 18,
                 emb_dim = 10, 
                 hidden_dim = 10,
                 num_layers = 3,
                 dropout = 0,
                 batch_norm = False,
                 bidirectional = False):
        super(LSTM, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_dim, emb_dim, padding_idx=0)
        self.encoder = torch.nn.LSTM(emb_dim, hidden_dim, num_layers, bidirectional = bidirectional, batch_first=True)
        if batch_norm:
            self.batch_norm = torch.nn.BatchNorm1d(hidden_dim)
        else:
            self.batch_norm = None
        self.dropout = torch.nn.Dropout(p=dropout)
        self.linear = torch.nn.Linear(hidden_dim, output_dim)
    def forward(self, input):
        out = self.embedding(input)
        out, _ = self.encoder(out)
        if self.batch_norm is not None:
            out = self.batch_norm(out.transpose(1, 2)).transpose(1, 2)
        out = self.dropout(out)
        out = self.linear(out)
        return out

### Загрузка датасета (предварительно скачаем сам архив)

In [18]:
docs = load_nerus("nerus_lenta.conllu.gz")

In [19]:
list_of_sent, list_of_tags = get_sent_tags(docs, 10000)

0it [00:00, ?it/s]

In [20]:
pos2idx, idx2pos = pos_dict(list_of_tags)

In [21]:
word2idx, idx2word = word_dict(list_of_sent)

In [22]:
train_data = NerusDataset(list_of_sent, list_of_tags, word2idx, pos2idx, train=True)
test_data = NerusDataset(list_of_sent, list_of_tags, word2idx, pos2idx, train=False)

### Обучение модели

In [23]:
loss_function = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam

Перебираемые параметры для анализа зависимости модели от них:

In [30]:
dim_layers_params = [10, 15, 20]
num_layers_params = [3, 5, 7]
dropout_params = [0, 0.25, 0.5]
batch_norm_params = [False, True]
len_dict_params = [10000, 25000, 50000]

Разные размеры слоя:

In [29]:
for dim in dim_layers_params:
    print(f'dim = {dim}')
    model = LSTM(vocab_dim=len(word2idx), emb_dim=dim, hidden_dim=dim, output_dim=len(pos2idx))
    model.to(device)
    writer = SummaryWriter(log_dir=f'tensorboard2/dim_{dim}')
    call = callback(writer, test_data, loss_function)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)

dim = 10
loss: 2.9790517902374267, acc: 0.022016221657395363
+----------+----------+---------------+
| Word     | True tag | Predicted tag |
+----------+----------+---------------+
| За       | ADP      | ADV           |
| семь     | NUM      | ADV           |
| месяцев  | NOUN     | ADV           |
| с        | ADP      | ADV           |
| начала   | NOUN     | ADV           |
| акции    | NOUN     | ADV           |
| в        | ADP      | ADV           |
| ней      | PRON     | ADV           |
| приняли  | VERB     | ADV           |
| участие  | NOUN     | ADV           |
| человек  | NOUN     | ADV           |
| в        | ADP      | ADV           |
| 52       | NUM      | ADV           |
| городах  | NOUN     | ADV           |
| России   | PROPN    | ADV           |
| ,        | PUNCT    | ADV           |
| они      | PRON     | ADV           |
| посетили | VERB     | ADV           |
| ,        | PUNCT    | ADV           |
| и        | CCONJ    | ADV           |
| детей    | NOUN  

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=300, train_loss=2.2013754844665527, val_loss=2.248526210784912, val_acc=0.2665121555328369


...

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1200, train_loss=0.9579474329948425, val_loss=0.9585205030441284, val_acc=0.7161065936088562


  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1500, train_loss=0.6939226984977722, val_loss=0.7853385519981384, val_acc=0.7757821083068848
loss: 0.7838173341751099, acc: 0.7850521206855774
+----------------+----------+---------------+
| Word           | True tag | Predicted tag |
+----------------+----------+---------------+
| Российский     | ADJ      | ADJ           |
| министр        | NOUN     | NOUN          |
| финансов       | NOUN     | NOUN          |
| и              | CCONJ    | CCONJ         |
| вице-премьер   | NOUN     | NOUN          |
| Антон          | PROPN    | PROPN         |
| Силуанов       | PROPN    | PROPN         |
| пообещал       | VERB     | VERB          |
| обратиться     | VERB     | VERB          |
| в              | ADP      | ADP           |
| Верховный      | ADJ      | ADJ           |
| суд            | NOUN     | NOUN          |
| Великобритании | PROPN    | PROPN         |
| .              | PUNCT    | PUNCT         |
+----------------+----------+---------------+


Чем больше размер слоя, тем лучше.

Разное число слоёв:

In [31]:
for num_layers in num_layers_params:
    print(f'num_layers = {num_layers}')
    model = LSTM(vocab_dim=len(word2idx), num_layers=num_layers)
    model.to(device)
    writer = SummaryWriter(log_dir=f'tensorboard2/num_layers_{num_layers}')
    call = callback(writer, test_data, loss_function)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)

num_layers = 3
loss: 2.852248439788818, acc: 0.05446118116378784
+-------------+----------+---------------+
| Word        | True tag | Predicted tag |
+-------------+----------+---------------+
| Игра        | NOUN     | ADJ           |
| начнется    | VERB     | ADJ           |
| в           | ADP      | ADJ           |
| 22          | NUM      | ADJ           |
| :           | PUNCT    | PART          |
| 00          | NUM      | PART          |
| по          | ADP      | PART          |
| московскому | ADJ      | PART          |
| времени     | NOUN     | PART          |
| .           | PUNCT    | PART          |
+-------------+----------+---------------+


epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=300, train_loss=2.2201104164123535, val_loss=2.2465128803253176, val_acc=0.26998841762542725


...

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1200, train_loss=2.1368587017059326, val_loss=2.1873823261260985, val_acc=0.2885283827781677


  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1500, train_loss=2.181196689605713, val_loss=2.1840294456481932, val_acc=0.2885283827781677
loss: 2.182241611480713, acc: 0.2885283827781677
+---------------+----------+---------------+
| Word          | True tag | Predicted tag |
+---------------+----------+---------------+
| Ранее         | ADV      | ADP           |
| в             | ADP      | NOUN          |
| воскресенье   | NOUN     | NOUN          |
| ,             | PUNCT    | NOUN          |
| 30            | ADJ      | NOUN          |
| сентября      | NOUN     | NOUN          |
| ,             | PUNCT    | NOUN          |
| объявил       | VERB     | NOUN          |
| ,             | PUNCT    | NOUN          |
| что           | SCONJ    | NOUN          |
| у             | ADP      | NOUN          |
| ВМС           | PROPN    | NOUN          |
| США           | PROPN    | NOUN          |
| «             | PUNCT    | NOUN          |
| есть          | VERB     | NOUN          |
| возможность   | NOUN     | NOUN         

Увеличивать число слоёв не надо.

Зависимость от dropout:

In [32]:
for dropout in dropout_params:
    print(f'dropout = {dropout}')
    model = LSTM(vocab_dim=len(word2idx), dropout=dropout)
    model.to(device)
    writer = SummaryWriter(log_dir=f'tensorboard2/dropout_{dropout}')
    call = callback(writer, test_data, loss_function)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)

dropout = 0
loss: 2.945477523803711, acc: 0.03244495764374733
+------------+----------+---------------+
| Word       | True tag | Predicted tag |
+------------+----------+---------------+
| В          | ADP      | PRON          |
| нем        | PRON     | PRON          |
| они        | PRON     | PRON          |
| опровергли | VERB     | PRON          |
| информацию | NOUN     | PRON          |
| о          | ADP      | PRON          |
| связи      | NOUN     | PRON          |
| с          | ADP      | PRON          |
| российской | ADJ      | PRON          |
| разведкой  | NOUN     | PRON          |
| и          | CCONJ    | PRON          |
| утверждали | VERB     | PRON          |
| ,          | PUNCT    | PRON          |
| что        | SCONJ    | PRON          |
| с          | ADP      | PRON          |
| визитом    | NOUN     | PRON          |
| и          | CCONJ    | PRON          |
| собирались | VERB     | PRON          |
| посмотреть | VERB     | PRON          |
| на         |

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=300, train_loss=2.2123637199401855, val_loss=2.2403217792510985, val_acc=0.26998841762542725


...

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1200, train_loss=1.635204553604126, val_loss=1.6136111688613892, val_acc=0.42989569902420044


  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1500, train_loss=1.5963246822357178, val_loss=1.5101778984069825, val_acc=0.444959431886673
loss: 1.4869454336166381, acc: 0.44553881883621216
+----------+----------+---------------+
| Word     | True tag | Predicted tag |
+----------+----------+---------------+
| Поставки | NOUN     | NOUN          |
| ЗРК      | PROPN    | NOUN          |
| начались | VERB     | NOUN          |
| в        | ADP      | NOUN          |
| конце    | NOUN     | NOUN          |
| сентября | NOUN     | NOUN          |
| .        | PUNCT    | PUNCT         |
+----------+----------+---------------+


Добавление dropout 0.25 заметно уменьшило loss, но также немного упала acc, нужен дополнительный анализ.

Добавление BatchNorm:

In [33]:
for batch_norm in batch_norm_params:
    print(f'batchnorm = {batch_norm}')
    model = LSTM(vocab_dim=len(word2idx), batch_norm=batch_norm)
    model.to(device)
    writer = SummaryWriter(log_dir=f'tensorboard2/batch_norm_{batch_norm}')
    call = callback(writer, test_data, loss_function)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)

batchnorm = False
loss: 2.8802488899230956, acc: 0.09327925741672516
+-------------+----------+---------------+
| Word        | True tag | Predicted tag |
+-------------+----------+---------------+
| Он          | PRON     | PROPN         |
| подчеркивал | VERB     | PROPN         |
| ,           | PUNCT    | PROPN         |
| что         | SCONJ    | PROPN         |
| весь        | DET      | PROPN         |
| план        | NOUN     | PROPN         |
| обойдется   | VERB     | PROPN         |
| бюджету     | NOUN     | PROPN         |
| в           | ADP      | ADP           |
| 25          | NUM      | ADP           |
| триллионов  | NOUN     | ADP           |
| рублей      | NOUN     | PROPN         |
| .           | PUNCT    | ADP           |
+-------------+----------+---------------+


epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=300, train_loss=2.1776533126831055, val_loss=2.244501762390137, val_acc=0.26998841762542725


...

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1200, train_loss=0.8375889658927917, val_loss=0.8608740854263306, val_acc=0.7097334861755371


  0%|          | 0/155 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

	 step=1500, train_loss=0.6483244895935059, val_loss=0.7195430016517639, val_acc=0.764194667339325
loss: 0.7100648617744446, acc: 0.7711471319198608
+-------------+----------+---------------+
| Word        | True tag | Predicted tag |
+-------------+----------+---------------+
| Украинская  | ADJ      | ADJ           |
| компания    | NOUN     | NOUN          |
| добивается  | VERB     | VERB          |
| ареста      | NOUN     | VERB          |
| европейских | ADJ      | VERB          |
| активов     | NOUN     | NOUN          |
| в           | ADP      | ADP           |
| целях       | NOUN     | NOUN          |
| .           | PUNCT    | PUNCT         |
+-------------+----------+---------------+


Очевидно, что добавление BatchNorm значительно улучшило модель.

Разные размеры словаря:

In [34]:
for len_dict in len_dict_params:
    print(f'len_dict = {len_dict}')
    list_of_sent, list_of_tags = get_sent_tags(docs, len_dict)
    pos2idx, idx2pos = pos_dict(list_of_tags)
    word2idx, idx2word = word_dict(list_of_sent)
    train_data = NerusDataset(list_of_sent, list_of_tags, word2idx, pos2idx, train=True)
    test_data = NerusDataset(list_of_sent, list_of_tags, word2idx, pos2idx, train=False)
    model = LSTM(vocab_dim=len(word2idx))
    model.to(device)
    writer = SummaryWriter(log_dir=f'tensorboard2/len_dict_{len_dict}')
    call = callback(writer, test_data, loss_function)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)
    trainer(count_of_epoch=10, 
            batch_size=64, 
            dataset=train_data,
            model=model, 
            loss_function=loss_function,
            optimizer = optimizer,
            callback=call)
    check_model(64, test_data, model, loss_function, idx2word, idx2pos)

len_dict = 10000


0it [00:00, ?it/s]

loss: 2.895909643173218, acc: 0.06179473549127579
+------------+----------+---------------+
| Word       | True tag | Predicted tag |
+------------+----------+---------------+
| В          | ADP      | PROPN         |
| ноябре     | NOUN     | PROPN         |
| 2017       | ADJ      | PROPN         |
| года       | NOUN     | PROPN         |
| героев     | NOUN     | PROPN         |
| фото       | NOUN     | PROPN         |
| обнаружили | VERB     | PROPN         |
| и          | PART     | PROPN         |
| на         | ADP      | PROPN         |
| других     | ADJ      | PROPN         |
| снимках    | NOUN     | PROPN         |
| из         | ADP      | PROPN         |
| фотобанка  | NOUN     | PROPN         |
| ,          | PUNCT    | PROPN         |
| составив   | VERB     | PROPN         |
| из         | ADP      | PROPN         |
| них        | PRON     | PROPN         |
| «          | PUNCT    | PROPN         |
| и          | CCONJ    | PROPN         |
| »          | PUNCT    | 

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/155 [00:00<?, ?it/s]

...

	 step=5700, train_loss=0.4182208180427551, val_loss=0.37907384753227236, val_acc=0.89217209815979
	 step=6000, train_loss=0.3642757534980774, val_loss=0.36859025597572326, val_acc=0.89217209815979


  0%|          | 0/780 [00:00<?, ?it/s]

	 step=6300, train_loss=0.33775076270103455, val_loss=0.3574409854412079, val_acc=0.8958008885383606
	 step=6600, train_loss=0.32299473881721497, val_loss=0.3458834254741669, val_acc=0.8994297385215759
	 step=6900, train_loss=0.31413009762763977, val_loss=0.3321727848052978, val_acc=0.9004665613174438


  0%|          | 0/780 [00:00<?, ?it/s]

	 step=7200, train_loss=0.32618218660354614, val_loss=0.3247833800315857, val_acc=0.9035769701004028
	 step=7500, train_loss=0.25204774737358093, val_loss=0.3175006139278412, val_acc=0.9051321744918823
	 step=7800, train_loss=0.2642120122909546, val_loss=0.30874947071075437, val_acc=0.9051321744918823
loss: 0.30874947071075437, acc: 0.9051321744918823
+--------------+----------+---------------+
| Word         | True tag | Predicted tag |
+--------------+----------+---------------+
| Когда        | SCONJ    | NUM           |
| он           | PRON     | PRON          |
| у            | ADP      | ADP           |
| вас          | PRON     | ADV           |
| в            | ADP      | ADP           |
| следственном | ADJ      | ADJ           |
| изоляторе    | NOUN     | NOUN          |
| ,            | PUNCT    | PUNCT         |
| тогда        | ADV      | ADV           |
| он           | PRON     | PRON          |
| будет        | AUX      | AUX           |
| давать       | VERB     | VE

Логично, что при большем количестве данных модель лучше, но дольше обучается.

## Выводы

Явный положительный эффект дало увеличение размера слоя, добавление BatchNorm и увеличение размера словаря.

Число слоёв, скорее всего, надо было увеличивать пропорционально росту размеров каждого слоя. А также пробовать комбинировать с другими улучшениями. Ведь в эксперименте был зафиксирован размер слоя 10, что, возможно, было слишком малым для числа слоёв 5 и 7. Либо маленького размера обучающей выборки не хватило для обучения более сложной модели.

При добавлении dropout, видимо, надо было дольше обучать модель (увеличить размер словаря или даже число эпох).

### Дополнительные выводы после просмотра tensorboard

BatchNorm заметно ускоряет обучение; увеличение размера слоя однозначно улучшает модель; dropout 0.5 слишком большой, 0.25 уменьшает loss, но почти не меняет acc, если зафиксировать размер выборки и число эпох обучения; увеличение размера словаря безоговорочно важно для качественного обучения модели; увеличение количества слоёв сильно усложняет модель, что ведёт к необходимости более долгого обучения (ближе к концу обучения модель с 5 слоями резко начала улучшаться, хотя до этого была похожа на модель с 7 слоями, а сначала даже модель с 3 слоями по графику loss на train не отличалась от 5 и 7 слоёв).